In [1]:
import numpy as np

In [2]:
demo = False
board_width = 0
if (demo):
    file = 'input_ex1.txt'
    board_width = 16
else:
    # Puzzle input
    file = 'input.txt'
    board_width = 150

In [3]:
file = open(file, 'r')
input = file.readlines()
file.close()

In [4]:
import sys

def print_board(command=None):

    original_stdout = sys.stdout # Save a reference to the original standard output

    with open('board_progress.txt', 'a') as f:
        sys.stdout = f # Change the standard output to the file we created.
        
        if command != None:
            print('Command: '+str(command))
        
        for i in range(len(board)):
            line = ""
            for j in range(board_width):
                if board[i,j] == 0:
                    line += " "
                elif board[i,j] == 1:
                    line += "."
                elif board[i,j] == 2:
                    line += "#"
                elif board[i,j] == 10:
                    line += ">"
                elif board[i,j] == 11:
                    line += "v"
                elif board[i,j] == 12:
                    line += "<"
                elif board[i,j] == 13:
                    line += "^"
            print(line)

        sys.stdout = original_stdout # Reset the standard output to its original value

In [5]:
# build grid
board = np.zeros((len(input)-2, board_width))
for i,line in enumerate(input[:-2]):
    for j,c in enumerate(line):
        if c == ".":
            board[i,j] = 1
        elif c == "#":
            board[i,j] = 2
#print_board()

In [6]:
# set starting point
current_ori = 10
if demo:
    current_pos = (0,8)
    board[0,8] = 10
else:
    current_pos = (0,50)
    board[0,50] = 10   

In [7]:
# parse path
path = [] 
number = ""
for c in input[-1].strip():
    if c == 'R' or c == 'L':
        path.append(int(number))
        number = ""
        path.append(c)
    else:
        number += c
path.append(int(number))

In [8]:
def find_new_target_pos(target_pos, ori):
    # cases: outside of matrix
    if target_pos[0] < 0 or ori == 13:
        for i in range(len(board),0,-1):
            new_pos = board[i-1,target_pos[1]]
            if new_pos in [1,10,11,12,13]:
                return (i-1,target_pos[1])
            elif new_pos == 2:
                return None
            else:
                continue
    elif target_pos[0] >= len(board) or ori == 11:
        for i in range(len(board)):
            new_pos = board[i,target_pos[1]]
            if new_pos in [1,10,11,12,13]:
                return (i,target_pos[1])
            elif new_pos == 2:
                return None
            else:
                continue
    elif target_pos[1] < 0 or ori == 12:
        for j in range(board_width,0,-1):
            new_pos = board[target_pos[0],j-1]
            if new_pos in [1,10,11,12,13]:
                return (target_pos[0],j-1)
            elif new_pos == 2:
                return None
            else:
                continue
    elif target_pos[1] >= board_width or ori == 10:
        for j in range(board_width):
            new_pos = board[target_pos[0],j]
            if new_pos in [1,10,11,12,13]:
                return (target_pos[0],j)
            elif new_pos == 2:
                return None
            else:
                continue
    return None

In [9]:
def go_single_step(source_pos, target_pos, ori):
    if target_pos[0] < 0 or target_pos[0] >= len(board) or target_pos[1] < 0 or target_pos[1] >= board_width:
        return find_new_target_pos(target_pos, ori)
    elif board[target_pos] in [1,10,11,12,13]:
        return target_pos
    elif board[target_pos] == 2:
        return None
    elif board[target_pos] == 0:
        return find_new_target_pos(target_pos, ori)
    return None

In [10]:
def go_x_steps(x, ori, pos):
    new_pos = pos
    for i in range(x):
        if ori == 10:
            pos = go_single_step(pos, (pos[0],pos[1]+1), ori)
        elif ori == 11:
            pos = go_single_step(pos, (pos[0]+1,pos[1]), ori)
        elif ori == 12:
            pos = go_single_step(pos, (pos[0],pos[1]-1), ori)
        elif ori == 13:
            pos = go_single_step(pos, (pos[0]-1,pos[1]), ori)
            
        if pos == None:
            break
        else:
            new_pos = pos
            board[new_pos] = ori
    return new_pos

In [11]:
def turn(c, ori):
    if c == 'R':
        ori += 1
    elif c == 'L':
        ori -= 1
    if ori == 14:
        ori = 10
    if ori == 9:
        ori = 13
    board[current_pos] = ori 
    return ori     

In [12]:
for c in path:
    #print("Command: "+str(c))
    if type(c) == int:
        current_pos = go_x_steps(c, current_ori, current_pos)
    else:
        current_ori = turn(c, current_ori)
    #print_board(c)

In [13]:
def compute_result(current_pos, current_ori):
    result = 1000 * (current_pos[0] + 1)
    result += 4 * (current_pos[1] + 1)
    result += current_ori - 10
    return result

In [14]:
compute_result(current_pos, current_ori)

77318